In [1]:
import sqlite3
import pandas as pd


connect_dw_imoveis = sqlite3.connect("../data/imoveis_dw.db")
data = pd.read_sql_query(r"""
    SELECT * FROM IMOVEIS
    """, 
    connect_dw_imoveis
    )

In [4]:
data.head()

,cidade,estado,area,num_quartos,num_banheiros,garagem,num_andares,aceita_animais,mobilia,valor_aluguel,valor_condominio,valor_iptu,valor_seguro_incendio
0,São Paulo,SP,70,2,1,1,7,1,1,3300,2065,211,42
1,São Paulo,SP,320,4,4,0,20,1,0,4960,1200,1750,63
2,Porto Alegre,RS,80,1,1,1,6,1,0,2800,1000,0,41
3,Porto Alegre,RS,51,2,1,0,2,1,0,1112,270,22,17
4,São Paulo,SP,25,1,1,0,1,0,0,800,0,25,11


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10692 entries, 0 to 10691
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   cidade                 10692 non-null  object
 1   estado                 10692 non-null  object
 2   area                   10692 non-null  int64 
 3   num_quartos            10692 non-null  int64 
 4   num_banheiros          10692 non-null  int64 
 5   garagem                10692 non-null  int64 
 6   num_andares            10692 non-null  int64 
 7   aceita_animais         10692 non-null  int64 
 8   mobilia                10692 non-null  int64 
 9   valor_aluguel          10692 non-null  int64 
 10  valor_condominio       10692 non-null  int64 
 11  valor_iptu             10692 non-null  int64 
 12  valor_seguro_incendio  10692 non-null  int64 
dtypes: int64(11), object(2)
memory usage: 1.1+ MB


In [2]:
from pycaret.regression import *

Desta vez será usado o *encoder* do próprio pycaret para tratar os dados categóricos. Assim, os dados de tipo objeto serão passados diretamente para o setup, sem tratamento prévio.

In [3]:
numeric_features = ['num_quartos', 'num_banheiros', 'garagem', 'num_andares', 'aceita_animais', 'mobilia']

s = setup(data,
         target='valor_aluguel',
         session_id=4447,
         numeric_features=numeric_features,
         log_experiment=True,
         experiment_name='exp_datapipe')

,Description,Value
0,session_id,4447
1,Target,valor_aluguel
2,Original Data,"(10692, 13)"
3,Missing Values,False
4,Numeric Features,10
5,Categorical Features,2
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(7484, 16)"


In [4]:
best = compare_models(fold=5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,96.1837,83291.1133,283.3089,0.9928,0.0816,0.0348,3.9760
xgboost,Extreme Gradient Boosting,100.0815,86879.8766,289.7545,0.9925,0.0884,0.0363,15.5800
gbr,Gradient Boosting Regressor,120.7946,86339.9550,288.3406,0.9925,0.0924,0.0448,0.5420
et,Extra Trees Regressor,84.9678,87070.9758,290.2706,0.9924,0.0847,0.0304,0.8700
lightgbm,Light Gradient Boosting Machine,104.7914,93581.3859,297.9628,0.9919,0.0832,0.0359,0.1860
rf,Random Forest Regressor,90.8478,103686.3917,318.3352,0.9910,0.0884,0.0316,1.1360
dt,Decision Tree Regressor,99.8058,214557.2210,461.2894,0.9813,0.1195,0.0370,0.0420
omp,Orthogonal Matching Pursuit,337.2300,301042.8736,546.8401,0.9737,0.1357,0.0898,0.0180
llar,Lasso Least Angle Regression,309.9641,326577.0489,554.0052,0.9715,0.1302,0.0888,0.0180
lr,Linear Regression,305.5010,459866.2688,606.9691,0.9600,0.1520,0.0993,0.7640


In [5]:
catboost = create_model('catboost', fold=5)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,102.0779,94104.1138,306.7639,0.9923,0.1008,0.0343
1,97.2462,105042.7947,324.1031,0.9909,0.0745,0.0326
2,100.0082,121439.0833,348.4811,0.9895,0.0857,0.0363
3,90.7873,53283.3477,230.8319,0.9954,0.0656,0.0329
4,90.7986,42586.2273,206.3643,0.9958,0.0816,0.0380
Mean,96.1837,83291.1133,283.3089,0.9928,0.0816,0.0348
SD,4.6608,30340.3295,55.0200,0.0025,0.0118,0.0021


In [8]:
tuned_cat = tune_model(catboost, fold=5)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,108.5188,99035.3139,314.6988,0.9919,0.1019,0.0364
1,107.9988,96321.9966,310.3579,0.9916,0.0790,0.0378
2,106.3905,122415.1680,349.8788,0.9894,0.0857,0.0378
3,100.8340,60222.2886,245.4023,0.9948,0.0713,0.0376
4,94.2081,37414.4114,193.4281,0.9963,0.0842,0.0394
Mean,103.5900,83081.8357,282.7532,0.9928,0.0844,0.0378
SD,5.4257,30282.0042,55.9687,0.0025,0.0101,0.0010


Finalizar e armazenar modelo

In [6]:
final_catboost = finalize_model(catboost)

In [8]:
save_model(final_catboost, '../model/modelo_final_cat')

Transformation Pipeline and Model Succesfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=['num_quartos',
                                                           'num_banheiros',
                                                           'garagem',
                                                           'num_andares',
                                                           'aceita_animais',
                                                           'mobilia'],
                                       target='valor_aluguel',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy=...
                 ('cluster_all', 'passthrough'),
                 ('dummy', 

### Fase de Testes

In [14]:
model_test = load_model('../model/modelo_final_cat')

Transformation Pipeline and Model Successfully Loaded


In [15]:
data_test = data.drop(['valor_aluguel'], axis=1)

In [16]:
data_test.head()

,cidade,estado,area,num_quartos,num_banheiros,garagem,num_andares,aceita_animais,mobilia,valor_condominio,valor_iptu,valor_seguro_incendio
0,São Paulo,SP,70,2,1,1,7,1,1,2065,211,42
1,São Paulo,SP,320,4,4,0,20,1,0,1200,1750,63
2,Porto Alegre,RS,80,1,1,1,6,1,0,1000,0,41
3,Porto Alegre,RS,51,2,1,0,2,1,0,270,22,17
4,São Paulo,SP,25,1,1,0,1,0,0,0,25,11


In [17]:
predict_model(model_test, data=data_test)

,cidade,estado,area,num_quartos,num_banheiros,garagem,num_andares,aceita_animais,mobilia,valor_condominio,valor_iptu,valor_seguro_incendio,Label
0,São Paulo,SP,70,2,1,1,7,1,1,2065,211,42,3292.475724
1,São Paulo,SP,320,4,4,0,20,1,0,1200,1750,63,5101.174913
2,Porto Alegre,RS,80,1,1,1,6,1,0,1000,0,41,2851.307466
3,Porto Alegre,RS,51,2,1,0,2,1,0,270,22,17,1121.215809
4,São Paulo,SP,25,1,1,0,1,0,0,0,25,11,776.024953
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,Porto Alegre,RS,63,2,1,1,5,0,1,402,24,22,1477.813171
10688,São Paulo,SP,285,4,4,4,17,1,0,3100,973,191,15008.282093
10689,Rio De Janeiro,RJ,70,3,3,0,8,0,1,980,332,78,6014.669890
10690,Rio De Janeiro,RJ,120,2,2,2,8,1,1,1585,279,155,12021.530043
